In [1]:
import imageio.v3 as iio
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tflite_runtime.interpreter import Interpreter
from PIL import Image

# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="your_model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load labels from labels.txt
with open('labels.txt', 'r') as f:
    labels = [line.strip() for line in f.readlines()]

def preprocess_frame(frame):
    """Preprocess the frame before passing it to the model"""
    img = Image.fromarray(frame)
    img = img.resize((input_details[0]['shape'][2], input_details[0]['shape'][1]))  # Resize according to model input
    img = np.array(img).astype(np.float32) / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def run_inference(frame):
    """Run inference on the frame and get predictions"""
    interpreter.set_tensor(input_details[0]['index'], frame)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data

def display_frame(frame, results):
    """Display the frame with annotations"""
    plt.imshow(frame)
    plt.title("TFLite Object Detection")

    # Process results (this is an example; adjust based on your TFLite model output format)
    for result in results:
        class_id = int(result[0])  # Assuming class ID is at index 0
        if class_id in [0, 2, 9]:  # Filter for person (0), car (2), traffic light (9)
            label = labels[class_id]
            plt.text(10, 10, f'Detected: {label}', color='red', fontsize=12)

    plt.show()

# Open video source (0 for webcam or provide path to video file)
cap = iio.get_reader("<video0>")  # Use "<video0>" for webcam

for frame in cap:
    # Preprocess frame for inference
    preprocessed_frame = preprocess_frame(frame)

    # Run inference
    results = run_inference(preprocessed_frame)

    # Display results
    display_frame(frame, results)

    # Break condition (e.g., you can add a way to exit the loop)
    if False:  # Replace with a condition to break the loop
        break


ModuleNotFoundError: No module named 'tflite_runtime'